In [ ]:
from PIL import Image, ImageDraw, ImageFont, ImageColor
import qrcode
import requests
from io import BytesIO
from typing import Optional, Literal, Union
from enum import Enum
import re

class PixQRCodeGenerator:
    class ColorOptions(str, Enum):
        BLACK = "black"
        WHITE = "white"
        PURPLE = "purple"
        BLUE = "blue"
        RED = "red"
        GREEN = "green"
        ORANGE = "orange"
        PINK = "pink"
        BROWN = "brown"
        GRAY = "gray"

        @classmethod
        def validate(cls, color: str) -> Union[str, tuple[int, int, int]]:
            c = color.strip()
            if c.lower() in cls._value2member_map_:
                return c.lower()
            if re.fullmatch(r'#?[0-9A-Fa-f]{6}', c):
                if not c.startswith('#'):
                    c = f'#{c}'
                return ImageColor.getrgb(c)
            raise ValueError(f"Invalid color '{color}'. Use one of: {', '.join(cls._value2member_map_)} or a HEX code like '#FFA500'")

    def __init__(self, pix_code: str):
        self.pix_code = pix_code

    def _load_logo(self, source: str) -> Image.Image:
        if source.startswith(("http://", "https://")):
            resp = requests.get(source, stream=True, timeout=5)
            resp.raise_for_status()
            resp.raw.decode_content = True
            img = Image.open(resp.raw)
        else:
            img = Image.open(source)
        return img.convert("RGBA")

    def _fit_text_font(self, draw: ImageDraw.ImageDraw, text: str, max_width: int, scale: float, font_name: str = "DejaVuSans.ttf", base_size: int = 40) -> ImageFont.FreeTypeFont:
        target = int(base_size * scale)
        for size in range(target, 9, -1):
            try:
                f = ImageFont.truetype(font_name, size)
                if draw.textlength(text, font=f) <= max_width:
                    return f
            except OSError:
                pass
        return ImageFont.load_default()

    def _extract_dominant_color(self, img: Image.Image) -> tuple[int, int, int]:
        thumb = img.convert("RGB")
        thumb.thumbnail((100, 100))
        colors = thumb.getcolors(thumb.width * thumb.height)
        return max(colors, key=lambda t: t[0])[1]

    def generate_qrcode_image(
        self,
        output_path: str = "pix_qrcode.png",
        logo_path: Optional[str] = None,
        logo_url: Optional[str] = None,
        logo_position: Literal["center", "top", "bottom"] = "center",
        qr_color: str = "auto",
        background_color: str = "white",
        title: Optional[str] = None,
        message: Optional[str] = None,
        qr_box_size: int = 10,
        size: Literal["small", "medium", "large"] = "medium",
        max_logo_ratio: float = 0.25,
    ) -> None:
        logo = None
        if logo_url:
            logo = self._load_logo(logo_url)
        elif logo_path:
            logo = self._load_logo(logo_path)

        if qr_color == "auto" and logo:
            qr_fill = self._extract_dominant_color(logo)
        else:
            qr_fill = self.ColorOptions.validate(qr_color)

        bg_fill = self.ColorOptions.validate(background_color)
        scale = {"small": 0.5, "medium": 1.0, "large": 2.0}[size]
        qr = qrcode.QRCode(error_correction=qrcode.constants.ERROR_CORRECT_H, box_size=int(qr_box_size * scale), border=4)
        qr.add_data(self.pix_code)
        qr.make(fit=True)
        qr_img = qr.make_image(fill_color=qr_fill, back_color=bg_fill).convert("RGB")

        w, h = qr_img.size
        extra = int(180 * scale) if (title or message or (logo and logo_position in ["top", "bottom"])) else 0
        final = Image.new("RGB", (w, h + extra), bg_fill)
        draw = ImageDraw.Draw(final)
        y = int(10 * scale)

        if title:
            ft = self._fit_text_font(draw, title, w - 20, scale)
            tw = draw.textlength(title, font=ft)
            draw.text(((w - tw) // 2, y), title, fill=qr_fill, font=ft)
            y += ft.size + int(10 * scale)

        if logo and logo_position == "top":
            lm = logo.copy()
            lm.thumbnail((int(w * max_logo_ratio),) * 2, Image.Resampling.LANCZOS)
            final.paste(lm, ((w - lm.width) // 2, y), lm)
            y += lm.height + int(10 * scale)

        final.paste(qr_img, (0, y))
        qr_y = y
        y += h

        if logo and logo_position == "bottom":
            lm = logo.copy()
            lm.thumbnail((int(w * max_logo_ratio),) * 2, Image.Resampling.LANCZOS)
            final.paste(lm, ((w - lm.width) // 2, y + int(10 * scale)), lm)
            y += lm.height + int(10 * scale)

        if message:
            fm = self._fit_text_font(draw, message, w - 20, scale)
            mw = draw.textlength(message, font=fm)
            draw.text(((w - mw) // 2, y + int(10 * scale)), message, fill=qr_fill, font=fm)

        if logo and logo_position == "center":
            lm = logo.copy()
            lm.thumbnail((int(w * max_logo_ratio),) * 2, Image.Resampling.LANCZOS)
            final.paste(lm, ((w - lm.width) // 2, qr_y + (h - lm.height) // 2), lm)

        final.save(output_path)
        print(f"[✔] QR Code salvo em: {output_path}")


In [ ]:
# Default values:
pix_code = "00020126920014br.gov.bcb.pix2570qrcodes.sulcredi.coop.br/v2/v3/at/c1c80f84-7400-47ae-bc96-6831bc417d505204000053039865802BR5925DOUBLE_EMPRESA_DE_COBRANC6008CURITIBA62070503***63040F59"
generator = PixQRCodeGenerator(pix_code)
LOGO="/home/agentkyo/Imagens/bet365-logo.png"

In [ ]:
generator.generate_qrcode_image(
  output_path="/home/agentkyo/Snap_n_ad/pix_qrcode_itau.png",
    logo_url="https://www.estilobordados.com.br/wp-content/uploads/2024/07/itau.png",
    logo_position="top",
    qr_color="#FF6200",
    background_color="white",
    title="Pague com Pix e receba 10% de Cashback",
    message="",
    size="medium"
)


In [ ]:
#Nubank Example

generator.generate_qrcode_image(
  output_path="/home/agentkyo/Snap_n_ad/pix_qrcode_nubank.png",
    logo_url="https://s2.glbimg.com/jSjr-G312XxUPZPlUoPKYusnrpI=/620x350/e.glbimg.com/og/ed/f/original/2021/05/17/novo_logo.png",
    logo_position="top",
    qr_color="#820AD1",
    background_color="white",
    title="",
    message="100 milhões de clientes.Nenhum deles na fila",
    size="medium"
)


In [ ]:
#Bet 365 Example -> Depostit R$50,00 - Caio Vinicius - 3**.***.***-00

generator.generate_qrcode_image(
    output_path="/home/agentkyo/Snap_n_ad/bet365_payer_data.png",
    logo_path="/home/agentkyo/Snap_n_ad/static/images/bet365.png",
    logo_position="top",
    qr_color="#14805E",
    background_color="white",
    title=None,
    message="Depósito R$50,00 - Caio Vinicius - 3**.***.***-00",
    size="large"
)


In [ ]:
#KTO Example 
generator.generate_qrcode_image(
    output_path="/home/agentkyo/Snap_n_ad/kto_example.png",
    logo_url="https://www.datocms-assets.com/119781/1715841828-kto-logo.webp",
    logo_position="bottom",
    qr_color="red",
    background_color="white",
    title="Know the Odds !",
    message="",
    size="medium"
)


In [ ]:
# Mercado Livre Example - With HexCode Color
generator.generate_qrcode_image(
    output_path="/home/agentkyo/Snap_n_ad/meli.png",
    logo_url="https://play-lh.googleusercontent.com/2uEptSnLOcFWnSjxLLIfGOWrf6lWSQ8pheDbGdzhqn0dcV93PBSrUkyjAKPgY7ejmA",
    logo_position="center",
    qr_color="#FFE600",
    background_color="white",
    title="Até 30% off",
    message="Aproveite o dia dos pais com ofertas imperdíveis!",
    size="medium"
)


In [ ]:
# Bradesco - With HexCode Color
generator.generate_qrcode_image(
    output_path="/home/agentkyo/Snap_n_ad/bradesco.png",
    logo_url="https://t.ctcdn.com.br/v-jWl8gzXbYv0sNkWB3ldTSrAhY=/1080x1080/smart/i620776.jpeg",
    logo_position="center",
    qr_color="#EE012B",
    background_color="white",
    title="Bradesco",
    message="O melhor banco para investir",
    size="medium"
)


In [ ]:
# Totvs Techfin - With HexCode Color
generator.generate_qrcode_image(
    output_path="/home/agentkyo/Snap_n_ad/totvsTechfin.png",
    logo_url="https://www.totvs.com/wp-content/uploads/2025/06/logo-totvs-itau-rebranding.png",
    logo_position="center",
    qr_color="#0D2D3D",
    background_color="white",
    title="Totvs Techfin",
    message="Segurança e Agilidade",
    size="medium"
)


In [ ]:
# Shellbox  - With HexCode Color
generator.generate_qrcode_image(
    output_path="/home/agentkyo/Snap_n_ad/shellbox.png",
    logo_url="https://play-lh.googleusercontent.com/FrSwdzib9XbPQO-DhGhzN1uAXwEbmDhZIZHS7nBCACPqFPYJW2_DA6qyFGJsVW7toQ",
    logo_position="center",
    qr_color="#DD1D21",
    background_color="white",
    title="#POSTO ABC - Bomba XYZ",
    message="42 Litros - Etanol - Valor R$ 200,00",
    size="medium"
)
